<a href="https://colab.research.google.com/github/LUNA-HYUN/HelmetDetectionV0/blob/main/HelmetDetectionV0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''본 코드는 Google Colab에서 GPU를 사용해 실행한다는 가정하에 작성했습니다.'''

# ==================================================================================
# 1. 환경 설정 및 데이터 준비 (Environment Setup & Data Preparation)
# ==================================================================================
print(">> [1. Setup] 필수 라이브러리 설치 및 환경 설정 중...")
!pip install ultralytics kagglehub -q

import os
import glob
import yaml
import cv2
import torch
import kagglehub
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from ultralytics import YOLO
from IPython.display import Image, display

# 하드웨어 가속(GPU) 설정 확인
DEVICE = 0 if torch.cuda.is_available() else 'cpu'
print(f"✅ 사용 장치(Device): {torch.cuda.get_device_name(0) if DEVICE == 0 else 'CPU'}")

# ----------------------------------------------------------------------------------
# 2. 데이터셋 다운로드 및 상세 분석 (Dataset Analysis)
# ----------------------------------------------------------------------------------
print("\n>> [2. Data] 데이터셋 다운로드 및 무결성 검사...")
try:
    dataset_path = kagglehub.dataset_download("muhammetzahitaydn/hardhat-vest-dataset-v3")
    print(f"📂 데이터셋 저장 경로: {dataset_path}")
except Exception as e:
    print(f"❌ 다운로드 실패: {e}")

# 폴더 구조 자동 탐색
images_root = os.path.join(dataset_path, 'images')
if not os.path.exists(images_root): images_root = dataset_path

subdirs = os.listdir(images_root)
train_dir = next((d for d in subdirs if 'train' in d), None)
val_dir = next((d for d in ['valid', 'val', 'validation'] if d in subdirs), None)
test_dir = next((d for d in subdirs if 'test' in d), None)

# [상세 정보 0] 폴더 구조 표시
print("\n📂 폴더 구조:")
def print_folder_structure(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 3 * (level)
        if level == 0:
            print(f"   📁 {os.path.basename(root)}/") # 기본 폴더 이름
        else:
            # 현재 디렉토리의 파일 수 계산
            file_count = len(files)
            dir_name = os.path.basename(root)
            if dir_name:
                print(f"{indent}   📁 {dir_name}/: {file_count}개 파일")

    # 데이터셋 경로의 루트에 있는 파일 처리 (예: README 또는 data.yaml)
    root_files = [f for f in os.listdir(startpath) if os.path.isfile(os.path.join(startpath, f))]
    if root_files:
        print(' ' * 3)
        for f in root_files:
            print(f"   📄 {f}")

print_folder_structure(dataset_path)

# [상세 정보 1] 이미지 수 확인
print("\n📊 [데이터셋 통계 - 이미지 수량]")
data_dirs = {'Train': train_dir, 'Valid': val_dir, 'Test': test_dir}
total_images = 0
for name, d in data_dirs.items():
    if d:
        count_jpg = len(glob.glob(os.path.join(images_root, d, '*.jpg')))
        count_png = len(glob.glob(os.path.join(images_root, d, '*.png')))
        count = count_jpg + count_png
        print(f"   - {name} Set: {count}장")
        total_images += count
print(f"   👉 총 이미지 수: {total_images}장")


# [상세 정보 2] 라벨 데이터 분포 분석 (클래스별 개수)
print("\n📊 [데이터셋 통계 - 클래스 분포]")
# 클래스 정의 (0:Helmet, 1:Vest, 2:Head, 3:Person)
class_names = {0: 'Helmet', 1: 'Vest', 2: 'Head', 3: 'Person'}
label_counts = Counter()

# Train 라벨 파일 전체 스캔
train_label_path = os.path.join(dataset_path, 'labels', train_dir) if os.path.exists(os.path.join(dataset_path, 'labels')) else os.path.join(dataset_path, 'train', 'labels')
if os.path.exists(train_label_path):
    label_files = glob.glob(os.path.join(train_label_path, '*.txt'))
    for lf in label_files:
        with open(lf, 'r') as f:
            for line in f:
                try:
                    cls_id = int(line.split()[0])
                    label_counts[cls_id] += 1
                except: pass

# 분포 출력
df_stats = pd.DataFrame.from_dict(label_counts, orient='index', columns=['Count'])
df_stats.index = df_stats.index.map(class_names)
print(df_stats.sort_values('Count', ascending=False))
print(f"   👉 Head(2)와 Helmet(0) 데이터가 풍부함 확인됨.")

# ----------------------------------------------------------------------------------
# 3. 데이터 설정 파일(YAML) 생성 (Configuration)
# ----------------------------------------------------------------------------------
print("\n>> [3. Config] 학습 설정 파일(data.yaml) 생성...")
yaml_path = '/content/safety_data.yaml'
yaml_content = {
    'path': dataset_path,
    'train': os.path.join('images', train_dir),
    'val': os.path.join('images', val_dir),
    'nc': 4,
    'names': ['Helmet', 'Vest', 'Head', 'Person']
}
with open(yaml_path, 'w') as f:
    yaml.dump(yaml_content, f, sort_keys=False)
print(f"✅ 설정 파일 생성 완료: {yaml_path}")


In [ ]:
# ----------------------------------------------------------------------------------
# 4. 모델 로드 및 학습 (Model Training)
# ----------------------------------------------------------------------------------
print("\n>> [4. Train] YOLOv8 모델 학습 시작...")
model = YOLO('yolov8s.pt')  # Pre-trained 모델 로드

# 학습 파라미터 설정 (첨부파일 파이프라인 참조)
project_name = 'safety_project_final'
results = model.train(
    data=yaml_path,
    epochs=20,          # 전체 데이터 반복 학습 횟수
    imgsz=640,          # 이미지 입력 크기
    batch=32,           # 배치 사이즈 (T4 GPU 최적화)
    patience=5,         # 조기 종료 (과적합 방지)
    workers=4,          # 데이터 로더 워커 수
    device=DEVICE,
    project='/content/runs/detect',
    name=project_name,
    exist_ok=True,
    verbose=True
)
print("✅ 모델 학습 완료!")

In [ ]:
# ----------------------------------------------------------------------------------
# 5. 학습 결과 분석 (Evaluation Visualization)
# ----------------------------------------------------------------------------------
print("\n>> [5. Eval] 학습 결과 시각화...")
results_dir = f'/content/runs/detect/{project_name}'

# 결과 그래프 (Loss, mAP)
results_img = os.path.join(results_dir, 'results.png')
if os.path.exists(results_img):
    print("📈 [학습 곡선 (Loss & mAP)]")
    display(Image(results_img, width=800))

# 혼동 행렬 (Confusion Matrix)
cm_img = os.path.join(results_dir, 'confusion_matrix.png')
if os.path.exists(cm_img):
    print("📉 [혼동 행렬 (Confusion Matrix)]")
    display(Image(cm_img, width=600))
else:
    print("⚠️ 혼동 행렬 이미지가 생성되지 않았습니다.")

print(f"\n✅ 최적 모델 저장 위치: {os.path.join(results_dir, 'weights', 'best.pt')}")

In [ ]:
# ==================================================================================
# 6. 추론 및 시각화 (Inference and Visualization)
# ==================================================================================
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from ultralytics import YOLO
import glob
import os

print(">> [6. Inference] 스마트 안전 모니터링 시스템 가동...")

# 1. 학습된 최적 모델 로드
try:
    best_model_path = '/content/runs/detect/safety_project_final/weights/best.pt'
    model = YOLO(best_model_path)
    print(f"✅ 모델 로드 성공: {os.path.basename(best_model_path)}")
except:
    print("❌ 모델을 찾을 수 없습니다. Part 1 학습을 먼저 완료해주세요.")
    model = None

# ----------------------------------------------------------------------------------
# 위험 구역(Danger Zone) 기반 4색 안전 판별 로직
# ----------------------------------------------------------------------------------
def draw_danger_zone(img, points):
    """이미지에 반투명한 위험 구역(Red Zone)을 시각화합니다."""
    overlay = img.copy()
    cv2.fillPoly(overlay, [points], (0, 0, 255)) # 빨간색 영역
    alpha = 0.3 # 투명도
    cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)
    cv2.polylines(img, [points], True, (0, 0, 255), 3) # 테두리

    # 구역 이름 표시
    m = cv2.moments(points)
    if m["m00"] != 0:
        cx, cy = int(m["m10"] / m["m00"]), int(m["m01"] / m["m00"])
        cv2.putText(img, "DANGER ZONE", (cx - 60, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

def is_in_zone(box, zone_points):
    """작업자가 위험 구역 내부에 있는지 판별합니다 (Point inside Polygon)."""
    x1, y1, x2, y2 = box
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2 # 작업자의 중심점
    return cv2.pointPolygonTest(zone_points, (cx, cy), False) >= 0

def run_safety_analysis(model, image_path):
    img = cv2.imread(image_path)
    if img is None: return
    h, w = img.shape[:2]

    # 1. 위험 구역 정의 (가상 시나리오: 화면 중앙 하단이 위험 구역)
    zone_pts = np.array([
        [int(w*0.2), int(h*0.3)], [int(w*0.8), int(h*0.3)],
        [int(w*0.9), int(h*0.9)], [int(w*0.1), int(h*0.9)]
    ], np.int32)
    draw_danger_zone(img, zone_pts)

    # 2. 모델 추론 (Confidence 0.25)
    results = model.predict(image_path, conf=0.25, verbose=False)
    if not results: return

    # 3. 객체 분석 및 상태 판정
    for box in results[0].boxes:
        cls_id = int(box.cls[0])
        cls_name = model.names[cls_id].lower()
        xyxy = box.xyxy[0].cpu().numpy()
        x1, y1, x2, y2 = map(int, xyxy)

        # 'Helmet'(안전) 또는 'Head'(위험) 클래스만 분석 대상
        has_helmet = 'helmet' in cls_name
        is_head = 'head' in cls_name

        if not (has_helmet or is_head): continue

        # 위치 확인: 구역 내부인가?
        in_zone = is_in_zone(xyxy, zone_pts)

        # --- [핵심] 4단계 색상 코드 (Color Code) ---
        if in_zone:
            if has_helmet:
                color, label = (0, 255, 0), "Authorized (In Zone)"  # 🟢 구역 내 승인됨
            else:
                color, label = (0, 0, 255), "CRITICAL: LEAVE ZONE!" # 🔴 구역 내 헬멧 미착용
        else:
            if has_helmet:
                color = (255, 0, 0) # 파란색 (OpenCV는 BGR)
                label = "Safe (Outside)"                            # 🔵 구역 외 안전
            else:
                color, label = (0, 255, 255), "Warning: Helmet?"    # 🟡 구역 외 주의

        # 시각화
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)
        (text_w, text_h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv2.rectangle(img, (x1, y1 - 25), (x1 + text_w, y1), color, -1)
        txt_color = (0, 0, 0) if color in [(0, 255, 255), (0, 255, 0)] else (255, 255, 255)
        cv2.putText(img, label, (x1, y1 - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.6, txt_color, 2)

    print(f"\n📸 분석 결과: {os.path.basename(image_path)}")
    cv2_imshow(img)

# 테스트 실행 (검증 데이터셋에서 랜덤 샘플링)
import random
dataset_path = '/kaggle/input/hardhat-vest-dataset-v3'
test_images = glob.glob(os.path.join(dataset_path, '**', 'images', '**', '*.jpg'), recursive=True)
# 다양한 케이스 확인을 위해 랜덤하게 3장 선택
sample_images = random.sample(test_images, 3) if test_images else []

if model and sample_images:
    for img_path in sample_images:
        run_safety_analysis(model, img_path)
else:
    print("⚠️ 테스트 이미지를 찾을 수 없습니다.")

In [ ]:
# ==================================================================================
# 7. 모델 저장 및 내보내기 (Export)
# ==================================================================================
print(">> [7. Export] 모델 변환 및 저장...")

try:
    if os.path.exists(best_model_path):
        # ONNX 포맷으로 변환 (CPU 모드 권장)
        model.export(format='onnx', device='cpu')

        print("\n✅ 모델 내보내기 성공!")
        print(f"   📦 ONNX 파일 위치: {best_model_path.replace('.pt', '.onnx')}")
        print(f"   💾 PT 파일 위치:   {best_model_path}")
        print("\n[안내] 왼쪽 파일 탐색기에서 해당 경로를 찾아 우클릭 -> 다운로드 하시면 됩니다.")
    else:
        print("⚠️ 저장할 모델 파일(best.pt)이 없습니다.")
except Exception as e:
    print(f"❌ 내보내기 중 오류 발생: {e}")

print("\n🎉 모든 프로젝트 과정이 성공적으로 완료되었습니다!")